In [1]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [2]:
# for index, row in df.iterrows():
# doc_id = row.doc_id
# import ast
# authorList = ast.literal_eval(row.authorList)
authorList = [11, 18, 80, 88, 64]
doc_id = 1

parameters = {
    'candidate': [2, 3, 4, 5],
    'samples': [320, 1600, 3200],
}

# print('doc_id: %3s actual author: %4s author_id: %s' % (str(doc_id), str(row.author_id), str(authorList)))

level = 'word'

for idxp, paralist in parameters.iteritems():
    if idxp == 'samples':
        for idxl, val in enumerate(paralist):

            authorList = [11, 18, 80, 88, 64]
            doc_id = 1

            candidate = len(authorList)

            test = idxp # change before run

            level = "word"

            iterations = 30

            dropout = 0.5

            samples = val

            dimensions = 200

            # loc = authorList.index(author_id)

            printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
            printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
            printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

            print("Current test: %s" % (str(printstate)))

            import UpdateDB as db

            case = db.checkOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions, samples = samples,
                                  iterations = iterations, dropout = dropout, test = test)

            if case == False:

                print("Running: %12s" % (str(printstate)))

                import StyloNeural as Stylo
                (labels_index, history, train_acc, val_acc, samples) = Stylo.getResults(
                    doc_id = doc_id, authorList = authorList[:], 
                    level = level, glove = '../../glove/', dimensions = dimensions, 
                    samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

                # for key, auth in labels_index.iteritems():
                #     if auth == author_id:
                #         loc = key

                # ans = predY[loc]

                import UpdateDB as db
                case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                             samples = samples, iterations = iterations, dropout = dropout, 
                                             train_acc = train_acc, val_acc = val_acc, test = test)

                # if case:
                #     ans = 0.0

                #     if (predY.tolist().index(max(predY)) == loc):
                #         ans = 1.0
                # 
                #     import UpdateDB as db
                #     case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                #                                  samples = samples, iterations = iterations, dropout = dropout, 
                #                                  accuracy = ans, test = str(test + 'binary'))

                # else:
                #     print("Skipped: %12s" % (str(printstate)))

                del Stylo

                from keras import backend as K
                K.clear_session()

                import time
                time.sleep(10)

                from IPython.display import clear_output
                clear_output()

            else:
                print("Skipped: %12s" % (str(printstate)))
            
        
    if idxp == 'candidate':
        for idxl, val in enumerate(paralist):

            authorList = [11, 18, 80, 88, 64]
            doc_id = 1

            candidate = val

            test = idxp # change before run

            level = "word"

            iterations = 30

            dropout = 0.5

            samples = 3200

            dimensions = 200

            # loc = authorList.index(author_id)

            printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
            printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
            printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

            print("Current test: %s" % (str(printstate)))

            import UpdateDB as db

            case = db.checkOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions, samples = samples,
                                  iterations = iterations, dropout = dropout, test = test)

            if case == False:

                print("Running: %12s" % (str(printstate)))

                import StyloNeural as Stylo
                (labels_index, history, train_acc, val_acc, samples) = Stylo.getResults(
                    doc_id = doc_id, authorList = authorList[:candidate+1], 
                    level = level, glove = '../../glove/', dimensions = dimensions, 
                    samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

                # for key, auth in labels_index.iteritems():
                #     if auth == author_id:
                #         loc = key

                # ans = predY[loc]

                import UpdateDB as db
                case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                             samples = samples, iterations = iterations, dropout = dropout, 
                                             train_acc = train_acc, val_acc = val_acc, test = test)

                # if case:
                #     ans = 0.0

                #     if (predY.tolist().index(max(predY)) == loc):
                #         ans = 1.0
                # 
                #     import UpdateDB as db
                #     case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                #                                  samples = samples, iterations = iterations, dropout = dropout, 
                #                                  accuracy = ans, test = str(test + 'binary'))

                # else:
                #     print("Skipped: %12s" % (str(printstate)))

                del Stylo

                from keras import backend as K
                K.clear_session()

                import time
                time.sleep(10)

                from IPython.display import clear_output
                clear_output()

            else:
                print("Skipped: %12s" % (str(printstate)))
                

Current test: doc_id = 1, candidate = 5, dimensions = 200, samples = 320, 
iterations = 30, dropout = 0.5, test = samples
Execution completed
Running: doc_id = 1, candidate = 5, dimensions = 200, samples = 320, 
iterations = 30, dropout = 0.5, test = samples


Using TensorFlow backend.


Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 279
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (39639, 2)
Author:    11  Size: 11064
Author:    18  Size: 11253
Author:    64  Size:  5106
Author:    80  Size:  6423
Author:    88  Size:  5793
Min: 5106
Max: 11253
Authors [11, 18, 64, 80, 88].
Found 1600 texts.
Found 1600 labels.
Found 31863 unique tokens.
Shape of data tensor: (1600, 1000)
Shape of label tensor: (1600, 5)
Done compiling.
Train on 1280 samples, validate on 320 samples
Epoch 1/30
1280/1280 [==============================] - 16s - loss: 1.5967 - acc: 0.2664 - val_loss: 1.5029 - val_acc: 0.2875
Epoch 2/30
1280/1280 [==============================] - 14s - loss: 1.4054 - acc: 0.3648 - val_loss: 1.2777 - val_acc: 0.4656
Epoch 3/30
1280/1280 [==============================] - 14s - loss: 1.0581 - acc: 0.5625 - val_loss: 1.0327 - val_acc: 0.6250
Epoch 4/30
1280/1280 

SystemExit: 1

In [3]:
%tb

SystemExit: 1